# Wizeline task for bootcamp

### Carla Garcia

# Import libraries

In [1]:
import pandas as pd

# Load data

In [9]:
dataset=pd.read_csv('.\data\sample.csv')

In [97]:
#Chunk big file
#dataset_profeco = pd.read_csv('.\profeco.zip', chunksize=1000000)

def read_chunk(columns):
    dataset_profeco = pd.read_csv('.\profeco.zip', chunksize=10000000)
    chunk_list = []  # append each chunk df here 
    i=0
    # Each chunk is in df format
    for chunk in dataset_profeco:  
        # perform data filtering 
        chunk_filter = chunk[columns]
        print(i)
        # Once the data filtering is done, append the chunk to list
        chunk_list.append(chunk_filter)
        i=i+1
    # concat the list into dataframe 
    df_concat = pd.concat(chunk_list)
    return df_concat

## Q1 Process

In [3]:
#Extraction
COLUMNS_Q1=['cadenaComercial']
dataset_Q1=dataset['cadenaComercial']
dataset_Q1=read_chunk(COLUMNS_Q1)
#Transformation
count_cadena_comercial_Q1=dataset_Q1.nunique()
del dataset_Q1

NameError: name 'dataset' is not defined

## Q2 Process

In [4]:
#Extraction
COLUMNS_Q2=['estado','producto','marca']
#dataset_Q2_little=dataset[COLUMNS_Q2]
dataset_Q2=read_chunk(COLUMNS_Q2)
#Transformation
top_10_products=dataset_Q2.groupby(COLUMNS_Q2,as_index=False).agg({'marca':'count'})
top_10_products_filter=top_10_products.sort_values(['estado','marca'],ascending=(True,False)),groupby('estado').head(10).reset_index(level=[0,1])
top_10_products_filter

0
1
2
3


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (5,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


4
5
6


ValueError: cannot insert marca, already exists

In [49]:
#Extraction
COLUMNS_Q2=['estado','producto','marca']
#dataset_Q2=dataset[COLUMNS_Q2]
dataset_Q2=pd.read_csv(r'.\top_10_raw_data.csv')
#Transformation
#group per state and product count.
top_10_products=dataset_Q2.groupby(['estado','producto'],group_keys=False)['producto'].count()
#work with sample
#top_10_products_filter=top_10_products.groupby('estado',group_keys=False).nlargest(10)
#top_10_products_filter
#Work with big file
#convert series to dataframe.
df=pd.DataFrame(top_10_products)
#rename column with counts to cuenta
df=df.rename(columns={'producto':'cuenta'})
#reset previous index
df=df.reset_index()
#sort values by state and cuenta
df=df.sort_values(['estado','cuenta'],ascending=(True,False))
#group by estado and only take the first 10
df_10=df.groupby('estado',group_keys=False).head(10)
df_10.head(50)
del dataset_Q2

,estado,producto,cuenta
397,AGUASCALIENTES,FUD,12005
302,AGUASCALIENTES,DETERGENTE P/ROPA,10188
520,AGUASCALIENTES,LECHE ULTRAPASTEURIZADA,9824
813,AGUASCALIENTES,SHAMPOO,9654
762,AGUASCALIENTES,REFRESCO,9481
301,AGUASCALIENTES,DESODORANTE,8859
467,AGUASCALIENTES,JABON DE TOCADOR,8517
214,AGUASCALIENTES,CHILES EN LATA,7946
930,AGUASCALIENTES,YOGHURT,7401
575,AGUASCALIENTES,MAYONESA,7173


## Q3 process

In [5]:
#Extraction
COLUMNS_Q3=['cadenaComercial','producto']
#dataset_Q3=dataset[COLUMNS_Q3]
dataset_Q3=read_chunk(COLUMNS_Q3)
#Transformation
cadena_per_product=dataset_Q3.groupby(['cadenaComercial']).producto.count()
cadena_most_monitoring_prod=cadena_per_product.nlargest(1).reset_index()
cadena_most_monitoring_prod['cadenaComercial'][0:]
del dataset_Q3

0
1
2
3


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (5,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


4
5
6


# Q1 How many commercial chains are monitored, and therefore, included in this database?


In [60]:
print(f"There are {count_cadena_comercial_Q1[0]} commercial chains which products are  monitored")

There are 705 commercial chains which products are  monitored


#  Q2 What are the top 10 monitored products by State?

In [50]:
path=r'.\top_10_per_state.csv'
df_10.to_csv('example.csv')
print(f"Top 10 products by state can be reviewed in {path}")

Top 10 products by state can be reviewed in .\top_10_per_state.csv


# Q3 Which is the commercial chain with the highest number of monitored products?

In [6]:
print(f" Comercial chain with highest number of monitored products is: {cadena_most_monitoring_prod['cadenaComercial'].values[0]}")

 Comercial chain with highest number of monitored products is: WAL-MART


In [7]:
cadena_most_monitoring_prod

,cadenaComercial,producto
0,WAL-MART,8643133


# Q4 Use the data to find an interesting fact

In [159]:
from folium import plugins
import folium
import sys
import pandas as pd
import datetime

#19.44244244	-99.1309882
dataset=pd.read_csv('.\data\sample.csv')
map_hooray = folium.Map(location=[19.4424, -99.1309882],zoom_start = 13) 
df_acc=dataset
print('Hola')
print(df_acc[df_acc['direccion']=='AV. LAGO DE GUADALUPE S/N ESQ. DE LAS VILLAS'])
# Ensure you're handing it floats
df_acc['latitud'] = df_acc['latitud'].astype(float)
df_acc['longitud'] = df_acc['longitud'].astype(float)

nan_rows = df_acc[df_acc['latitud'].isnull()]
print(nan_rows)

#converting it to datetime format

df_acc['fechaRegistro']=pd.to_datetime(df_acc['fechaRegistro'])
#formatting the index
heat_df=df_acc
heat_df['Timestamp']=heat_df['fechaRegistro'].dt.strftime('%d/%m/%Y, %H:%M:%S')
heat_df2=heat_df[['latitud','longitud','Timestamp']]
date_strings=heat_df2['Timestamp'].unique()
heat_df2=heat_df2.dropna()
#heat_df['fechaRegistro'] = heat_df['fechaRegistro'].astype('datetime64[us]').astype(datetime.datetime).strftime('%d/%m/%Y, %H:%M:%S')
heat_data = [[[row['latitud'],row['longitud']] for index,row in heat_df2[heat_df2['Timestamp'] == fecha].iterrows()] for fecha in date_strings]
#print(date_strings)
print(heat_data)
# Plot it on the map
hm = plugins.HeatMapWithTime(heat_data,auto_play=True,max_opacity=0.8)
hm.add_to(map_hooray)
# Display the map
map_hooray
#print(date_strings)
#%reset -f


Hola
                                     producto  \
9                          PAN BLANCO BOLILLO   
10                           HARINA HOT CAKES   
11                            PASTA PARA SOPA   
12                            GALLETAS DULCES   
13                            GALLETAS DULCES   
..                                        ...   
774                                     ARROZ   
775                               CARNE POLLO   
776                               CARNE CERDO   
777                                 CARNE RES   
778  CHULETA DE RES/CHULETA COSTILLA/CHULETON   

                                         presentacion                 marca  \
9                                               PIEZA                   S/M   
10                                       CAJA 800 GR.  PRONTO TRADICIONALES   
11                          PAQUETE 200 GR. SPAGHETTI                 VESTA   
12                             CAJA 652 GR. RICANELAS                GAMESA   
13         

[[[19.29699, -99.125417], [19.29699, -99.125417], [19.29699, -99.125417], [19.29699, -99.125417], [19.29699, -99.125417], [19.29699, -99.125417], [19.29699, -99.125417], [19.29699, -99.125417], [19.29699, -99.125417]], [[19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.366903, -99.181344], [19.3

In [150]:

'''# Create weight column, using date
#heat_df['Weight'] = df_acc['fechaRegistro']
heat_df['Weight'] = heat_df['Weight'].astype(float)
heat_df = heat_df.dropna(axis=0, subset=['latitud','longitud', 'Weight'])

# List comprehension to make out list of lists
heat_data = [[[row['latitud'],row['Longitude']] for index, row in heat_df[heat_df['Weight'] == i].iterrows()] for i in range(0,13)]

# Plot it on the map
hm = plugins.HeatMapWithTime(heat_data,auto_play=True,max_opacity=0.8)
hm.add_to(map_hooray)
# Display the map
map_hooray'''
heat_df2.head()
heat_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1229 entries, 0 to 1998
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitud    1229 non-null   float64
 1   longitud   1229 non-null   float64
 2   Timestamp  1229 non-null   object 
dtypes: float64(2), object(1)
memory usage: 38.4+ KB


In [119]:
import pandas as pd

#dataset=pd.read_csv('.\data\sample.csv')
dataset['fechaRegistro'].nunique()
dataset.head()
dataset['fechaRegistro'].nunique()
COLUMNS_Q3=['fechaRegistro']
dataset_Q3=dataset[COLUMNS_Q3]
dataset=read_chunk(COLUMNS_Q3)
print('Result ',dataset['fechaRegistro'].nunique())


NameError: name 'read_chunk' is not defined

# Q5 What are the lessons learned from this excersice?

# Q6 Can you identify other ways to approach this problem? Explain.